In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.interpolate import LinearNDInterpolator
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.svm import SVR
DAYS_IN_SECONDS = 24 * 60 * 60

In [ ]:
class TrajectoryInterpolator:
    delta_xy_avg = 5 # nominal average distance between points in x and y, 5 km
    delta_z_avg = 5 # nominal average distance between points in z (1 day * 5 km / day)
    s_avg = None
    x_avg = None
    y_avg = None
    z_avg = None
    date_0 = None

    def __init__(self, **kwargs):
        """ Set parameters """
        self.__dict__.update(kwargs)

    def fit(self, df):
        self.read_trajectories(df)
        self.normalize_coordinates()
        self.prepare_train_data()
        self.train_interpolators()

    def read_trajectories(self, df):
        self.date_0 = pd.Timestamp(df.d.min())
        self.traj_x = [] # km
        self.traj_y = [] # km
        self.traj_t = [] # days
        self.traj_u = [] # km / day
        self.traj_v = [] # km / day
        self.traj_s = [] # km / day
        for _, g in df.groupby('g'):
            g_sorted = g.sort_values('d')
            g_days = (g.d - self.date_0).dt.total_seconds() / DAYS_IN_SECONDS
            self.traj_x.append(g_sorted.x.values / 1e3)
            self.traj_y.append(g_sorted.y.values / 1e3)
            self.traj_t.append(g_days.values)
            u = np.diff(g_sorted.x.values) / 1e3 / np.diff(g_days.values)
            v = np.diff(g_sorted.y.values) / 1e3 / np.diff(g_days.values)
            s = np.hypot(u, v)
            self.traj_u.append(u)
            self.traj_v.append(v)
            self.traj_s.append(s)

    def normalize_coordinates(self):
        self.s_avg = np.mean(np.hstack(self.traj_s))
        self.x_avg = np.mean(np.hstack(self.traj_x))
        self.y_avg = np.mean(np.hstack(self.traj_y))

        traj_z = [t * self.s_avg for t in self.traj_t]  # time coordinate expressed in km
        self.z_avg = np.mean(np.hstack(traj_z))

        self.traj_xn = [(x - self.x_avg) / self.delta_xy_avg for x in self.traj_x]
        self.traj_yn = [(y - self.y_avg) / self.delta_xy_avg for y in self.traj_y]
        self.traj_zn = [(z - self.z_avg) / self.delta_z_avg for z in traj_z]
        self.traj_un = [u * self.delta_z_avg / self.s_avg / self.delta_xy_avg for u in self.traj_u]
        self.traj_vn = [v * self.delta_z_avg / self.s_avg / self.delta_xy_avg for v in self.traj_v]

    def prepare_train_data(self):
        inputs = np.column_stack([
            np.hstack([i[:-1] for i in self.traj_xn]),
            np.hstack([i[:-1] for i in self.traj_yn]),
            np.hstack([i[:-1] for i in self.traj_zn]),
        ])
        targets_u = np.hstack([i for i in self.traj_un])
        targets_v = np.hstack([i for i in self.traj_vn])

        # Split the data into training and testing sets (80% train, 20% test)
        self.X_train, self.X_test, self.y_u_train, self.y_u_test, self.y_v_train, self.y_v_test = train_test_split(
            inputs, targets_u, targets_v, test_size=0.1)

    def train_interpolators(self):
        self.lndi_u = LinearNDInterpolator(self.X_train, self.y_u_train, rescale=True)
        self.lndi_v = LinearNDInterpolator(self.X_train, self.y_v_train, rescale=True)
        y_u_test_pred = self.lndi_u(self.X_test)
        y_v_test_pred = self.lndi_v(self.X_test)

        self.mean_squared_error = mean_squared_error(
            self.y_u_test[np.isfinite(y_u_test_pred)],
            y_u_test_pred[np.isfinite(y_u_test_pred)])
        self.r2_score = r2_score(
                  self.y_u_test[np.isfinite(y_u_test_pred)],
                  y_u_test_pred[np.isfinite(y_u_test_pred)])
        # Compute metrics
        print("Mean Squared Error:", self.mean_squared_error)
        print("R^2 Score:", self.r2_score)

    def normalize_start_coordinates(self, x0, y0, t0, t1, delta_t):
        x0n = (x0 / 1e3 - self.x_avg) / self.delta_xy_avg
        y0n = (y0 / 1e3 - self.y_avg) / self.delta_xy_avg
        t0_days = (t0 - self.date_0).total_seconds() / DAYS_IN_SECONDS
        z0 = t0_days * self.s_avg
        z0n = (z0 - self.z_avg) / self.delta_z_avg

        t1_days = (t1 - self.date_0).total_seconds() / DAYS_IN_SECONDS
        z1 = t1_days * self.s_avg
        z1n = (z1 - self.z_avg) / self.delta_z_avg

        delta_zn = delta_t.seconds / DAYS_IN_SECONDS * self.s_avg / self.delta_z_avg

        z0n = np.full(x0.shape, z0n)
        return x0n, y0n, z0n, z1n, delta_zn

    def grow_trajectories(self, x0n, y0n, z0n, z1n, dzn):
        traj_i_x = []
        traj_i_y = []
        traj_i_z = []
        time_len = int((z1n - z0n[0] + dzn) / dzn)
        for i in range(time_len):
            traj_i_x.append(x0n.copy())
            traj_i_y.append(y0n.copy())
            traj_i_z.append(z0n.copy())
            inputs0 =  np.column_stack([x0n, y0n, z0n])
            u_pred = self.lndi_u(inputs0)
            v_pred = self.lndi_v(inputs0)
            u_pred_norm = dzn * u_pred
            v_pred_norm = dzn * v_pred

            x0n += u_pred_norm
            y0n += v_pred_norm
            z0n += dzn
        traj_i_x, traj_i_y, traj_i_z = list(map(np.array, (traj_i_x, traj_i_y, traj_i_z)))        
        return traj_i_x, traj_i_y, traj_i_z

    def back2si(self, traj_i_x, traj_i_y, time_start, time_stop, time_step):
        traj_x = 1e3 * (traj_i_x * self.delta_xy_avg + self.x_avg)
        traj_y = 1e3 * (traj_i_y * self.delta_xy_avg + self.y_avg)
        traj_t = pd.date_range(time_start, time_stop, freq=time_step).to_series()
        return traj_x, traj_y, traj_t


In [ ]:
# Read feather file and convert to a DataFrame usable by pysida
feather_file = 'points.feather'
#points = pd.read_feather(feather_file)
#df = points[['image_id', 'trajectory_id', 'time', 'x', 'y', 'corr']].copy().rename(columns={'image_id':'i', 'trajectory_id': 'g', 'time':'d', 'x':'x', 'y':'y', 'corr':'q'})
df = pd.read_feather(feather_file)
print(df.shape)
df.head()

In [ ]:
# plot all points between two days
sub_df0 = df[(df.d > datetime(2020, 1, 15)) & (df.d < datetime(2020, 1, 16))]

radius = 1e5

center_x = 0.5e6
center_y = -0.5e6

#center_x = 0
#center_y = -0.6e6

#center_x = 0.6e6
#center_y = 0

dist = np.hypot(sub_df0.x - center_x, sub_df0.y - center_y) < radius
plt.scatter(sub_df0.x, sub_df0.y, c=dist, s=0.1, alpha=0.1)
plt.colorbar()
end_point_g = sub_df0[dist].g.unique()
print(sub_df0.shape, sub_df0.g.unique().size, end_point_g.size)

In [ ]:
sub_df = df[(df.d > datetime(2020, 1, 1)) & (df.d < datetime(2020, 1, 21))]

In [ ]:
# Select ONLY with end of trajectroies in these two days
disk_df = sub_df[sub_df['g'].isin(end_point_g)]
print(disk_df.shape, disk_df.g.unique().size)

In [ ]:
ti = TrajectoryInterpolator()
ti.fit(disk_df)

In [ ]:
_ = plt.hist(np.hstack(ti.traj_xn), bins=50)
_ = plt.hist(np.hstack(ti.traj_yn), bins=50, alpha=0.5)
_ = plt.hist(np.hstack(ti.traj_zn), bins=50, alpha=0.5)
plt.show()

_ = plt.hist(np.hstack(ti.traj_un), bins=50)
_ = plt.hist(np.hstack(ti.traj_vn), bins=50, alpha=0.5)
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
for x, y, z in zip(ti.traj_xn, ti.traj_yn, ti.traj_zn):
    plt.plot(z, y, 'k.-', alpha=0.1)
plt.xticks(np.arange(-24, 18, 1))
plt.grid()
plt.show()

In [ ]:
# create initial points for interpolation
time_start = pd.Timestamp('2020-01-03')
time_stop = pd.Timestamp('2020-01-20')
time_step = pd.Timedelta('6H')

x0df = disk_df.x
y0df = disk_df.y

res = 10000
x0lim = np.percentile(x0df, [1,99]) + np.array([0, res])
y0lim = np.percentile(y0df, [1,99]) + np.array([0, res])

x0grd, y0grd = np.meshgrid(np.arange(*x0lim, res), np.arange(*y0lim, res))
x0, y0 = x0grd.flatten(), y0grd.flatten()

plt.plot(x0df, y0df, '.', alpha=0.1)
plt.plot(x0, y0, '.')


x0n, y0n, z0n, z1n, dzn = ti.normalize_start_coordinates(x0, y0, time_start, time_stop, time_step)

In [ ]:
traj_ixn, traj_iyn, traj_izn = ti.grow_trajectories(x0n, y0n, z0n, z1n, dzn)
traj_x, traj_y, traj_t = ti.back2si(traj_ixn, traj_iyn, time_start, time_stop, time_step)

In [ ]:
for _, g in disk_df.groupby('g'):
    g_sorted = g.sort_values('d')
    plt.plot(g_sorted.d, g_sorted.x, 'k.-', alpha=0.1)

for x in traj_x.T:
    plt.plot(traj_t, x, 'r.-', alpha=0.1)

In [ ]:
# find finite starting points and triangulate
gpi = np.isfinite(traj_x[0])
t0 = Triangulation(traj_x[0, gpi], traj_y[0, gpi])

margin = 20000
xlim = np.nanpercentile(traj_x, [0, 100]) + np.array([-margin , margin])
ylim = np.nanpercentile(traj_y, [0, 100]) + np.array([-margin , margin])

# plot the advected triangulation
fig, axs = plt.subplots(1, 1, figsize=(10, 10))
plt.triplot(t0, color='gray', alpha=0.2)
for i in range(len(traj_x)):
    plt.plot(traj_x[i, gpi], traj_y[i, gpi], 'k.', alpha=0.1)
    trp = plt.triplot(traj_x[i, gpi], traj_y[i, gpi], t0.triangles, color='blue')
    axs.set_aspect('equal')
    axs.set_xlim(xlim)
    axs.set_ylim(ylim)
    plt.savefig(f'triangulation_step_{i:03d}.png', dpi=150)
    trp[0].remove()
    fig.canvas.draw()
    #plt.show()
    #break


In [ ]:
import glob
from PIL import Image

# Get all PNG files matching the pattern
png_files = sorted(glob.glob('triangulation_step_*.png'))

# Create list to store images
images = []

# Load all images
for file in png_files:
    img = Image.open(file)
    images.append(img)

# Save as GIF animation
if images:
    images[0].save('triangulation_animation.gif', 
                   save_all=True, 
                   append_images=images[1:], 
                   duration=200,  # milliseconds per frame
                   loop=0)  # infinite loop
    print(f"Created GIF animation with {len(images)} frames")
else:
    print("No PNG files found matching pattern 'triangulation_step_*.png'")